In [1]:
from geopy.geocoders import Nominatim
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pytz
import json

def get_lat_lon(city_name):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"위치를 찾을 수 없습니다: {city_name}")

def get_weather_info(city):
    try:
        lat, lon = get_lat_lon(city)
    except ValueError as e:
        print(e)
        return None

    location = Point(lat, lon)
    kst = pytz.timezone('Asia/Seoul')
    today = datetime.now(kst).date()
    yesterday = today - timedelta(days=1)
    start = datetime.combine(yesterday, datetime.min.time())
    end = datetime.combine(today, datetime.min.time()) + timedelta(days=1)

    data = Daily(location, start, end)
    data = data.fetch()

    if data.empty:
        print("데이터를 가져오지 못했습니다. 날짜 범위 또는 위치를 확인하세요.")
        return None

    yesterday_data = data.iloc[0] if len(data) >= 1 else None
    today_data = data.iloc[1] if len(data) >= 2 else None

    return json.dumps({
        "yesterday": {
            "avg_temp": yesterday_data['tavg'],
            "max_temp": yesterday_data['tmax'],
            "min_temp": yesterday_data['tmin'],
            "precipitation": yesterday_data['prcp'],
            "wind_speed": yesterday_data['wspd']
        },
        "today": {
            "avg_temp": today_data['tavg'],
            "max_temp": today_data['tmax'],
            "min_temp": today_data['tmin'],
            "precipitation": today_data['prcp'],
            "wind_speed": today_data['wspd'],
            "temp_difference": today_data['tmax'] - today_data['tmin']
        },
        "comparison": {
            "temp_max_difference": today_data['tmax'] - yesterday_data['tmax'],
            "temp_min_difference": today_data['tmin'] - yesterday_data['tmin']
        }
    })

In [8]:
import openai
import json
import os
import time

# OpenAI API를 사용하기 위한 클라이언트 객체를 생성합니다.
client = openai.OpenAI(api_key="")
ASSISTANT_ID = 'asst_tUcre9CS6mNDuLm90VEahIZU'

city = input("날씨 정보를 확인할 도시 이름을 입력하세요: ")

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"{city}의 날씨는 어제와 비교해서 어때? 뭐를 입어야 할까?",
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=ASSISTANT_ID,
)

tool_outputs = []

for tool in run.required_action.submit_tool_outputs.tool_calls:
    if tool.function.name == "get_weather_info":
        function_args = json.loads(tool.function.arguments)
        function_response = get_weather_info(function_args["city"])
        tool_outputs.append({
            "tool_call_id": tool.id,
            "output": function_response
        })

if tool_outputs:
    try:
        run = client.beta.threads.runs.submit_tool_outputs_and_poll(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    except Exception as e:
        print("Failed to submit tool outputs:", e)
else:
    print("No tool outputs to submit.")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages.data[1].content[0].text.value)
    print(messages.data[0].content[0].text.value)
else:
    print(run.status)

서울시 용산구의 날씨는 어제와 비교해서 어때? 뭐를 입어야 할까?
서울시 용산구의 오늘 날씨는 어제에 비해 더 덥습니다. 오늘의 평균 기온은 24.6도로, 어제보다 1.2도 높아졌습니다. 또한, 최고 기온은 30.4도에 이르는데, 이는 어제보다 4.8도나 높아졌습니다. 그래서 오늘은 어제보다 더 가벼운 옷차림을 추천드립니다. 

하지만, 일교차가 10.7도에 이르니, 이런 큰 일교차에 대비하시기 위해서는 얇은 외투나 가디건을 준비하시는 것도 좋을 것 같습니다.

또한, 오늘은 비가 조금 오는데, 어제보다 강한 풍속에 주의해야 합니다. 우산을 챙기시는 것을 잊지 마세요.


In [9]:
# 새 메시지를 추가하여 대화를 이어가기
new_message_content = input("새로운 메시지를 입력하세요: ")
new_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=new_message_content,
)

new_run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=ASSISTANT_ID,
)

if new_run.status == 'completed':
    new_messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(new_messages.data[0].content[0].text.value)
else:
    print(new_run.status)

오늘의 최저 기온은 19.7도로 선선한 편입니다. 그래서 아침과 저녁에는 조금 쌀쌀할 수 있어요. 또한, 일교차가 10.7도로 큰 편이니, 긴 바지를 입는 것이 더 적절해 보입니다. 하지만 낮에는 최고 기온이 30.4도로 상당히 더워지니, 통풍이 잘 되는 소재의 긴 바지를 선택하시면 좋을 것 같습니다.


In [1]:
import openai
import json
from geopy.geocoders import Nominatim
from meteostat import Point, Daily
from datetime import datetime, timedelta
import pytz

client = openai.OpenAI(api_key="")

def get_lat_lon(city_name):
    geolocator = Nominatim(user_agent="weather_app")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError(f"위치를 찾을 수 없습니다: {city_name}")

def get_weather_info(city):
    try:
        lat, lon = get_lat_lon(city)
    except ValueError as e:
        print(e)
        return None

    location = Point(lat, lon)
    kst = pytz.timezone('Asia/Seoul')
    today = datetime.now(kst).date()
    yesterday = today - timedelta(days=1)
    start = datetime.combine(yesterday, datetime.min.time())
    end = datetime.combine(today, datetime.min.time()) + timedelta(days=1)

    data = Daily(location, start, end)
    data = data.fetch()

    if data.empty:
        print("데이터를 가져오지 못했습니다. 날짜 범위 또는 위치를 확인하세요.")
        return None

    yesterday_data = data.iloc[0] if len(data) >= 1 else None
    today_data = data.iloc[1] if len(data) >= 2 else None

    return json.dumps({
        "yesterday": {
            "avg_temp": yesterday_data['tavg'],
            "max_temp": yesterday_data['tmax'],
            "min_temp": yesterday_data['tmin'],
            "precipitation": yesterday_data['prcp'],
            "wind_speed": yesterday_data['wspd']
        },
        "today": {
            "avg_temp": today_data['tavg'],
            "max_temp": today_data['tmax'],
            "min_temp": today_data['tmin'],
            "precipitation": today_data['prcp'],
            "wind_speed": today_data['wspd'],
            "temp_difference": today_data['tmax'] - today_data['tmin']
        },
        "comparison": {
            "temp_max_difference": today_data['tmax'] - yesterday_data['tmax'],
            "temp_min_difference": today_data['tmin'] - yesterday_data['tmin']
        }
    })


assistant = client.beta.assistants.create(
    instructions="You are a weather assistant named '코디 예보'. When asked about today's weather, you search and provide the current weather information and compare it with yesterday's weather from fuctions. Based on this comparison, you give clothing recommendations. For example, if it's hot than advise the short clothes.And if today is significantly colder than yesterday, advise the user to wear warm clothes than yesterday.기본적으로 오늘 온도가 30도 이상이면 엄청 더운거고, 20도 이상이면 더운거고, 10~20도는 선선한거야.10도 아래는  쌀쌀하고, 0도 아래면 너무 추운거니까 그에 맞는 옷차림 추천해줘.그리고 일교차가 10도 이상 크면 조심하라고 해줘.어제랑 비교해서 기온차가 많이 나면, 더 덥다, 더 추우니까 옷 잘 챙겨라 이런 식으로 조언해줘.",    
    model="gpt-4",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_weather_info",
                "description": "Get yesterday and today's weather data for clothing recommendations",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "The name of the city to get weather information for"
                        }
                    },
                    "required": ["city"]
                }
            }
        }
    ]
)

city = input("날씨 정보를 확인할 도시 이름을 입력하세요: ")

thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"{city}의 날씨는 어제와 비교해서 어때? 뭐를 입어야 할까?",
)

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)

tool_outputs = []

for tool in run.required_action.submit_tool_outputs.tool_calls:
    if tool.function.name == "get_weather_info":
        function_args = json.loads(tool.function.arguments)
        function_response = get_weather_info(function_args["city"])
        tool_outputs.append({
            "tool_call_id": tool.id,
            "output": function_response
        })

if tool_outputs:
    try:
        run = client.beta.threads.runs.submit_tool_outputs_and_poll(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    except Exception as e:
        print("Failed to submit tool outputs:", e)
else:
    print("No tool outputs to submit.")

if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages.data[1].content[0].text.value)
    print(messages.data[0].content[0].text.value)
else:
    print(run.status)

requires_action


Can you provide clothing recommendations in Korean based on yesterday's and today's weather in 서울시 용산구?
어제와 오늘의 서울시 용산구 날씨를 비교해 보았습니다.

어제의 평균 기온은 25.1도에 최고 기온은 29.1도, 최저 기온은 21.4도였습니다. 그리고 강수량은 4.6mm였어요.

오늘의 평균 기온은 23.4도에 최고 기온은 25.6도를 예상하고 있고, 최저 기온은 20.5도로 예상되고 있네요. 강수량도 9.8mm 예상되고 있어요. 

비교적으로 오늘은 어제보다 약간 추운 날씨입니다. 
특히 일교차가 대략 5.1도로 어제보다 큰 편이니까 감기 조심하세요.

옷차림은 연한 외투와 긴 바지를 추천합니다. 
비가 올 것 같으니 우산을 꼭 챙기시는 것이 좋을 것 같아요. 
기온 차가 많아 춥거나, 감기에 걸리지 않도록 항상 옷을 따뜻하게 입고 건강을 유의하시기 바랍니다.


In [9]:
# 새 메시지를 추가하여 대화를 이어가기
new_message_content = input("새로운 메시지를 입력하세요: ")
new_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=new_message_content,
)

new_run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

if new_run.status == 'completed':
    new_messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(new_messages.data[0].content[0].text.value)
else:
    print(new_run.status)

그렇습니다, 오늘의 풍속은 17.8km/h로 상당히 강한 편입니다. 바람막이나 외투를 준비하시는 것을 추천드리며, 모자나 스카프로 목과 머리를 따뜻하게 감싸는 것도 잊지 마세요. 특히 강한 바람 속에서 우산을 사용하면 휘날릴 수 있으니, 비올때는 방수 도토리를 이용하는 것이 좋을 것 같습니다.
